In [ ]:
import re
import sys
import json
import time
import traceback
import logging
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED
from datetime import datetime
from queue import Queue
from hashlib import sha256
import asyncio
import bson.json_util
from io import BytesIO
from bson import ObjectId
from collections import namedtuple
from typing import Generator, NamedTuple, List, Optional
from queue import Queue
import re

import pymongo
from kipp.decorator import timer
from kipp.utils import setup_logger
from pymongo import MongoClient
from tweepy import API, OAuthHandler
import requests
from bs4 import BeautifulSoup
from minio import Minio


logger = setup_logger("jav")
logger.setLevel(logging.DEBUG)
executor = ThreadPoolExecutor(max_workers=10)


# sys.path.append(r'/Users/laisky/repo/laisky/ramjet/ramjet/settings')
sys.path.append(r"/opt/configs/ramjet")
sys.path
import prd

# mongodb
mongo = MongoClient(
    f"mongodb://{prd.MONGO_ADMIN_USER}:{prd.MONGO_ADMIN_PASSWD}@{prd.MONGO_HOST}:{prd.MONGO_PORT}",
)
col_actress = mongo["jav"]["actress"]
col_movies = mongo["jav"]["movies"]


# add index to db
# notes.create_index([("post_id", pymongo.ASCENDING)], unique=True)

# minio
s3cli: Minio = Minio(
    endpoint=prd.S3_MINIO_ADDR,
    access_key=prd.S3_KEY,
    secret_key=prd.S3_SECRET,
    secure=True,
)

In [ ]:
actress_index_url = "https://onejav.com/actress/"


class Actress(NamedTuple):
    id: ObjectId
    name: str
    name_english: Optional[str]
    url: str


class Movie(NamedTuple):
    actress_id: ObjectId
    name: str
    description: str
    img_url: str
    tags: List[str]


def gen_actress() -> Generator[Actress, None, None]:
    n_page = 1
    n_retry = 0
    while True:
        url = f"https://onejav.com/actress/?page={n_page}"
        resp = requests.get(url)
        if resp.status_code == 500:
            if n_retry < 3:
                time.sleep(3)
                logger.warn(f"gen_actress retry {url=}, {n_retry=}")
                n_retry += 1
                continue
            else:
                n_retry = 0
                n_page += 1
                continue


        if resp.status_code != 200:
            logger.info(f"gen_actress exit for {url=} [{resp.status_code}]{resp.text}")
            return

        soup = BeautifulSoup(resp.text, "html.parser")
        for card in soup.select(".container .card"):
            name = (
                card.select_one(".card-header .card-header-title")
                .decode_contents()
                .strip()
            )
            url = (
                f"https://onejav.com{card.select_one('a').attrs['href']}?page="
            )
            logger.debug(f"yield actress {name}")

            actress = save_actress(name, url)
            yield actress

        n_page += 1


reg_name = re.compile('([\w ]+)(?:<[^>]+>(\w+).*)?')

def save_actress(name, url, name_english=None) -> Actress:
    """save actress to db, return actress_id"""
    logger.info(f"save actress {name}")

    # extract name
    # name = 'Jun Suehiro <small class="text-muted ml-1">末広純</small>'
    [(name_english, name)] = reg_name.findall(name)
    name = name.strip()
    name_english = name_english.strip() or None
    name = name or name_english

    docu = {"name": name, "url": url}
    name_english and docu.update({'name_english': name_english})

    col_actress.update_one(
        {"name": name},
        {"$set": docu},
        upsert=True,
    )

    docu = col_actress.find_one({"name": name})
    assert docu is not None, f"can not find actress {name}"
    return Actress(name=name, url=url, id=docu['_id'], name_english=name_english)


def replace_image_url(img_url) -> str:
    """download image and upload to s3, return new url"""
    resp = requests.get(img_url)
    assert resp.status_code == 200

    img_content = resp.content
    digest = sha256(img_content).hexdigest()
    objkey = f"jav/{digest[:2]}/{digest[2:4]}/{digest}.png"

    # check whether image exists
    new_img_url = f"{prd.S3_SERVER}/public/{objkey}"
    if requests.head(new_img_url).status_code == 200:
        return new_img_url

    logger.info(f"upload image to s3: {objkey}")
    s3cli.put_object(
        bucket_name="public",
        object_name=objkey,
        data=BytesIO(img_content),
        length=len(img_content),
        content_type="image/png",
    )

    return new_img_url


def gen_movies(actress: Actress):
    n_page = 1
    while True:
        url = f"{actress.url}{n_page}"
        resp = requests.get(url)
        if resp.status_code == 500:
            if n_retry < 3:
                time.sleep(3)
                logger.warn(f"gen_movies retry {url=}, {n_retry=}")
                n_retry += 1
                continue
            else:
                n_retry = 0
                n_page += 1
                continue

        if resp.status_code != 200:
            logger.info(f"gen_movies exit since for {url=} [{resp.status_code}]{resp.text}")
            return

        soup = BeautifulSoup(resp.text, "html.parser")
        for card in soup.select(".container .card"):
            img_url = card.select_one(".image").attrs["src"]
            name = card.select_one(".title a").decode_contents().strip()
            description = (
                card.select_one(".has-text-grey-dark").decode_contents().strip()
            )
            tags = [ele.decode_contents().strip() for ele in card.select(".tags .tag")]

            img_url = replace_image_url(img_url)

            movie = Movie(
                actress_id=actress.id,
                name=name,
                description=description,
                img_url=img_url,
                tags=tags,
            )
            yield movie

        n_page += 1


def save_movie(movie: Movie):
    logger.info(f"save movie {movie.name}")
    col_movies.update_one(
        {"name": movie.name},
        {
            "$set": {
                "actress_id": movie.actress_id,
                "name": movie.name,
                "description": movie.description,
                "img_url": movie.img_url,
                "tags": movie.tags,
            }
        },
        upsert=True,
    )

In [ ]:
@timer
def _run_gen_actress(q: Queue[Optional[Actress]]):
    for actress in gen_actress():
        q.put(actress)


@timer
def _run_gen_movies(
    id_: str, upstream: Queue[Optional[Actress]], downstream: Queue[Optional[Movie]]
):
    while True:
        actress = upstream.get()
        if actress is None:
            upstream.put(None)
            logger.info(f"_run_gen_movies[{id_}] exit")
            return

        for movie in gen_movies(actress):
            downstream.put(movie)


@timer
def _run_save_movie(id_: str, upstream: Queue[Optional[Movie]]):
    while True:
        movie = upstream.get()
        if movie is None:
            upstream.put(None)
            logger.info(f"_run_save_movie[{id_}] exit")
            return

        save_movie(movie)


@timer
def main():
    actress_queue = Queue(maxsize=10)
    movie_queue = Queue(maxsize=10)

    fs_step1 = []
    fs_step2 = []
    fs_step3 = []
    fs_step1.append(executor.submit(_run_gen_actress, actress_queue))
    for i in range(10):
        fs_step2.append(executor.submit(_run_gen_movies, str(i), actress_queue, movie_queue))
        fs_step3.append(executor.submit(_run_save_movie, str(i), movie_queue))

    wait(fs_step1, return_when=ALL_COMPLETED)
    actress_queue.put(None)
    wait(fs_step2, return_when=ALL_COMPLETED)
    movie_queue.put(None)
    wait(fs_step2, return_when=ALL_COMPLETED)


main()

In [ ]:
actress_index_url = "https://onejav.com/actress/Arina%20Hashimoto?page=1"

resp = requests.get(actress_index_url)
assert resp.status_code == 200

In [ ]:
soup = BeautifulSoup(resp.text, "html.parser")
card = soup.select(".container .card")[0]
img_url = card.select_one(".image").attrs["src"]
name = card.select_one(".title a").decode_contents().strip()
description = card.select_one(".has-text-grey-dark").decode_contents().strip()
tags = [ele.decode_contents().strip() for ele in card.select(".tags .tag")]




# for card in soup.select(".container .card"):
#     actress = card.select(".card-header .card-header-title").decode_contents()
#     print(actress)


In [ ]:
def replace_image_url(img_url) -> str:
    """download image and upload to s3, return new url
    """
    resp = requests.get(img_url)
    assert resp.status_code == 200

    img_content = resp.content
    digest = sha256(img_content).hexdigest()
    objkey = f"public/jav/{digest[:2]}/{digest[2:4]}/{digest}.png"

    s3cli.put_object(
        bucket_name="public",
        object_name=objkey,
        data=BytesIO(img_content),
        length=len(img_content),
        content_type="image/png",
    )

    return f"{prd.S3_SERVER}/public/{objkey}"